In [ ]:
!pip install konlpy
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 50.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.utils import to_categorical

# 데이터 불러오기
try:
    urllib.request.urlretrieve("https://raw.githubusercontent.com/tonykorea99/Spam-alart/main/dd.csv", filename='dd.csv')
    data = pd.read_csv('dd.csv', encoding='utf-8')
    data = data.sample(frac=1, random_state=42)
    print('총 샘플의 수:', len(data))
except Exception as e:
    print('에러 발생:', e)

# 레이블과 메일 내용이 담긴 v1, v2 열만 필요하며, unnamed 2-4 열은 삭제
data = data[['v1', 'v2']]

# 레이블 값을 0(ham), 1(spam)으로 변경
data['v1'] = data['v1'].replace(['ham', 'spam'], [0, 1])

# 중복 데이터 제거
data.drop_duplicates(subset=['v2'], inplace=True)
print('총 샘플의 수:', len(data))

# 스팸과 일반 메시지를 각각 1000개씩 뽑아내기
spam_data = data[data['v1'] == 1].head(1000)
ham_data = data[data['v1'] == 0].head(1000)

# 불용어 정의 (필요시 추가)
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과','ifg', '와', '네', '들', '듯', '지', '임', '게', '만', '되', '음', '면']

# 형태소 분리 및 토크나이징
def preprocess_text(text):
    okt = Okt()
    tokens = okt.morphs(text)
    tokens = [word for word in tokens if not word in stopwords]
    return ' '.join(tokens)

spam_data['v2'] = spam_data['v2'].apply(preprocess_text)
ham_data['v2'] = ham_data['v2'].apply(preprocess_text)

# 스팸과 일반 메시지 데이터 합치기
final_data = pd.concat([spam_data, ham_data], axis=0)

# 데이터 섞기
final_data = final_data.sample(frac=1, random_state=42)

# 레이블 및 데이터 분리
y = final_data['v1'].values
X_train, X_test, y_train, y_test = train_test_split(final_data['v2'], y, test_size=0.2, random_state=42)

# 토크나이저 및 패딩
max_len = 3039  # 최대 길이 설정
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_tokenized, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_tokenized, maxlen=max_len, padding='post')

# BiLSTM 모델 정의
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_len))
model_lstm.add(Bidirectional(LSTM(128)))
model_lstm.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
model_lstm.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.1)

# 모델 평가
loss, accuracy = model_lstm.evaluate(X_test_padded, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

총 샘플의 수: 13549
총 샘플의 수: 13477
Epoch 1/5
45/45 [==============================] - 22s 363ms/step - loss: 0.3766 - accuracy: 0.8417 - val_loss: 0.0640 - val_accuracy: 0.9812
Epoch 2/5
45/45 [==============================] - 15s 347ms/step - loss: 0.0137 - accuracy: 0.9979 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 3/5
45/45 [==============================] - 15s 328ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 4/5
45/45 [==============================] - 14s 322ms/step - loss: 5.1332e-04 - accuracy: 1.0000 - val_loss: 0.0093 - val_accuracy: 0.9937
Epoch 5/5
13/13 [==============================] - 2s 80ms/step - loss: 3.7123e-04 - accuracy: 1.0000
Accuracy: 100.00%


In [ ]:
print("Selected Data - Spam Count:", len(spam_data))
print("Selected Data - Ham Count:", len(ham_data))

Selected Data - Spam Count: 1000
Selected Data - Ham Count: 1000


In [ ]:
print("첫 번째 스팸 메시지:")
print(spam_data['v2'].values[0])

# 첫 번째 일반 메시지 출력
print("\n첫 번째 일반 메시지:")
print(ham_data['v2'].values[0])

첫 번째 스팸 메시지:
[ Web 발신 ] @( 광고 ) 김영기 고객 님 ifg@ifg @ ★ 전북 은행 이용 중 고객 님 께 드리는 특별한 혜택 ★ @ ① 무 방문 , 무 서류 , 무 보증 @ ② 전화 신청 만으로 당일 입금 @ ③ 고금 리카 드론 , 현금 서비스 , * 금융 권 대출 대 환 ifg@ifg @ ★ 전북 은행 언 택트 금융 센터 @ ☎ ****-**** 담당 직원 조 유미 과장 ifg@ifg @ □ 대상 : 기존 전북 은행 에서 대출 또는 신용카드 보유 고객 님 @ 상품 : 스피드 론 * @ 한도 : 최대 * 천만원 이내 @ 대상 : 연 소득 ** 백만원 이상 ( KCB 결정 연 소득 또는 건강 보험료 환산 소득 ) @( KCB , NICE ) CB 점수 구간 ** 구간 내 인자 @ 금리 : 최저 연 *.**%~ 최고 연 **%(* 개월 변동 금리 , 대출 기간 * 년 , 연 소득 * 천만원 ,**.*.** 기준 ) @ 기준금리 : 시장 기준금리 ( 금융채 AA +)* 개월 변동 : 연 *.**% 가산 금리 : 연 *.**~**.**( 신용등급 별 차등 ) @ 대출 기간 : 최소 * 년 ~ 최대 * 년 ( 연 단위 ) @ 상환 방법 : 원리금 균등 분할 상환 @ 중도 상환 수수료 :*.*% 대출 실행 후 * 개 월경 시 면제 이자 납부 시기 : 매월 이자 납입 일후 취 @ 금리인하 요구 권 : 대상 인지세 : 비 과세 @ ▷ 계약 체결 전 상품 설명 서 및 약관 읽어 보시기 바랍니다 @ ▷ 연체 시 지연 배상금 률 ( 약정 이자율 +*%, 최고 연 **%) 발생 하며 채무자 재산 / 신용 상의 불이익 ( 기한 이익 상실 등 ) 발생 할 수 있습니다 @ ▷ 상환 능력 비해 대 출금 과도 할 경우 개인 신용 평점 하락 할수있으며 , 따라 금융 거래 관련 된 불이익 발생 할수있습니다 . @ ▷ 일정 기간 원리금 연체 될 경우 계약 만료 도래 전 모든 원리금 변제 하는 의무 발생 할수있습니다 @ ▷ 금융기관 신용 관리 대상 고객 등 당 행및 보증 기관

In [ ]:
# 입력한 문장이 스미싱인지 일반 메시지인지 판별하는 함수
def classify_smishing(input_text):
    input_text = preprocess_text(input_text)
    encoded = tokenizer.texts_to_sequences([input_text])
    pad_tokens = pad_sequences(encoded, maxlen=max_len)
    smishing_score = float(model_lstm.predict(pad_tokens))

    if smishing_score > 0.5:
        print("{:.2f}% 확률로 스미싱입니다.".format(smishing_score * 100))
    else:
        print("{:.2f}% 확률로 일반 메시지입니다.".format((1 - smishing_score) * 100))

# 사용자로부터 텍스트 입력 받기
user_input_text = input("텍스트를 입력하세요: ")

# 스미싱 여부 판단 수행
classify_smishing(user_input_text)